## 图像变换

    transforms.Pad(padding,fill=0,padding_mode='constant')

- 功能

    对图像边缘进行填充

- padding

    设置填充大小
    - a:上下左右填充a个像素
    - (a,b) 上下填充b个像素，左右填充a个像素
    - (a,b,c,d) 左：a  上：b  右：c  下：d
    
- padding_mode

    填充模式
    
    - 1. constant  像素值由fill指定
    
    - 2. edge  像素值由图像边缘像素得到
    
    - 3. reflect  镜像填充，最后一个像素不镜像
    
    - 4. symmetric  镜像填充，最后一个像素填充

- fill

    填充的像素值
   

    代码实现：
        
        # Pad
        transforms.Pad(padding=32, fill=(255, 0, 0), padding_mode='constant'),
        transforms.Pad(padding=(8, 64), fill=(255, 0, 0), padding_mode='constant')
        transforms.Pad(padding=(8, 16, 32, 64), fill=(255, 0, 0), padding_mode='constant')
        transforms.Pad(padding=(8, 16, 32, 64), fill=(255, 0, 0), padding_mode='symmetric')

    transforms.ColorJitter(brightness=0,contrast=0,saturation=0,hue=0)

- 功能

    调整亮度，对比度，饱和度和色相

- brightness

    亮度调整因子
    
    - a: [max(1-a,a),a+1]中选择
    - (a,b): [a,b]中选择

- contrast

    对比度参数

- saturation

    饱和度参数
    
- hue

    色度
    - a: [-a,a+1]中选择   0=<a<=0.5
    - (a,b): [a,b]中选择   -0.5=<a<b<=0.5

    代码实现：
    
        # 2 ColorJitter
        transforms.ColorJitter(brightness=0.5)  亮度降低
        transforms.ColorJitter(contrast=0.5)    对比度下降
        transforms.ColorJitter(saturation=0.5)  饱和度下降
        transforms.ColorJitter(hue=0.3)         色度降低

    transforms.RandomGrayscale(num_output_channels,p=0.1)

- 功能

    依概率对对图像进行灰度化
    
- num_output_channels

    输出通道依据输入图像只能设置为1或3

- p为概率值


        transforms.Grayscale(num_output_channels)
    - 概率为1的RandomGrayscale

    代码实现：
    
        # 3 Grayscale
        # transforms.Grayscale(num_output_channels=3)
        

    transforms.RandomAffine(degrees,translate=None,scale=None,shear=None,resample=False,fillcolor=0)

- 功能

    对图像进行仿射变换（旋转，平移，缩放，错切和翻转）
    
- degrees

    旋转角度,必须设置

- translate

    平移区间设置：（a,b）
    
- scale:

    缩放比列，以面积为单位，取值（0,1）

- 填充颜色设置

- shear

    错切角度设置：
    - a 仅在x轴错切，角度在（-a,a）之间
    - （a,b） a设置x轴角度，b设置y轴角度
    - （a,b,c,d）a,b设置x轴角度，c,d设置y轴角度
    
- resample

    重采样方式：NEAREST,BILINEAR,BICUBIC

    代码实现：
        
        # 4 Affine
        transforms.RandomAffine(degrees=30)
        transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), fillcolor=(255, 0, 0))
        transforms.RandomAffine(degrees=0, scale=(0.7, 0.7))
        transforms.RandomAffine(degrees=0, shear=(0, 0, 0, 45))
        transforms.RandomAffine(degrees=0, shear=90, fillcolor=(255, 0, 0))

    transforms.RandomErasing(p=0.5,scale=(0.02,0.33),ratio=(0.3,3.3),value=0,inplace=False)

- 功能

    对图像进行随机遮挡
    
- p

    概率值，执行该操作的概率
    
- scale

    遮挡区域的面积
    
- ratio

    遮挡区域的长宽比
    
- value

    设置遮挡区域的像素值

- 注意：

    遮挡的输入必须是tensor，所以必须放在totensor后面

    代码实现：
    
        # 5 Erasing
        transforms.ToTensor(), ##必须是张量以后才能执行遮挡操作
        transforms.RandomErasing(p=1, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=(254/255, 0, 0))
        transforms.RandomErasing(p=1, scale=(0.02, 0.33), ratio=(0.3, 3.3), value='1234')

    transforms.Lambda(lambd)
    
- 功能

    用户自定义lambd方法
    
- lambd

    匿名函数
    
    lambda [arg1,arg2,...,argn]:expression
    
    例如：
    
        # 5 TenCrop
        transforms.TenCrop(112, vertical_flip=False),
        transforms.Lambda(lambda crops: torch.stack([(transforms.ToTensor()(crop)) for crop in crops]))

## transforms的选择操作

    transforms.RandomChoice([transforms1,transforms2,transforms3])

- 功能

    从一系列transforms的方法中随机选择一个


    transforms.RandomApply([transforms1,transforms2,transforms3],p=0.5)

- 功能

    依概率执行一组transforms操作

    transforms.RandomOrder([transforms1,transforms2,transforms3])

- 功能

    对一组transforms操作打乱顺序

    代码实现：
    
        # 1 RandomChoice
         transforms.RandomChoice([transforms.RandomVerticalFlip(p=1), transforms.RandomHorizontalFlip(p=1)]),

        # 2 RandomApply
         transforms.RandomApply([transforms.RandomAffine(degrees=0, shear=45, fillcolor=(255, 0, 0)),
                                 transforms.Grayscale(num_output_channels=3)], p=0.5),
        # 3 RandomOrder
         transforms.RandomOrder([transforms.RandomRotation(15),
                                 transforms.Pad(padding=32),
                                 transforms.RandomAffine(degrees=0, translate=(0.01, 0.1), scale=(0.9, 1.1))]),

In [2]:
import os
import numpy as np
import torch
import random
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
#from tools.my_dataset import RMBDataset
#from tools.common_tools import transform_invert

In [ ]:
'''
数据增强实例：
'''

def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)


set_seed(1)  # 设置随机种子

# 参数设置
MAX_EPOCH = 10
BATCH_SIZE = 1
LR = 0.01
log_interval = 10
val_interval = 1
rmb_label = {"1": 0, "100": 1}


# ============================ step 1/5 数据 ============================
split_dir = os.path.join("..", "..", "data", "rmb_split")  ##注意重设路径
train_dir = os.path.join(split_dir, "train")
valid_dir = os.path.join(split_dir, "valid")

norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]


train_transform = transforms.Compose([
    transforms.Resize((224, 224)),

    # 1 Pad
    # transforms.Pad(padding=32, fill=(255, 0, 0), padding_mode='constant'),
    # transforms.Pad(padding=(8, 64), fill=(255, 0, 0), padding_mode='constant'),
    # transforms.Pad(padding=(8, 16, 32, 64), fill=(255, 0, 0), padding_mode='constant'),
    # transforms.Pad(padding=(8, 16, 32, 64), fill=(255, 0, 0), padding_mode='symmetric'),

    # 2 ColorJitter
    # transforms.ColorJitter(brightness=0.5),
    # transforms.ColorJitter(contrast=0.5),
    # transforms.ColorJitter(saturation=0.5),
    # transforms.ColorJitter(hue=0.3),

    # 3 Grayscale
    # transforms.Grayscale(num_output_channels=3),

    # 4 Affine
    # transforms.RandomAffine(degrees=30),
    # transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), fillcolor=(255, 0, 0)),
    # transforms.RandomAffine(degrees=0, scale=(0.7, 0.7)),
    # transforms.RandomAffine(degrees=0, shear=(0, 0, 0, 45)),
    # transforms.RandomAffine(degrees=0, shear=90, fillcolor=(255, 0, 0)),

    # 5 Erasing
    # transforms.ToTensor(),
    # transforms.RandomErasing(p=1, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=(254/255, 0, 0)),
    # transforms.RandomErasing(p=1, scale=(0.02, 0.33), ratio=(0.3, 3.3), value='1234'),

    # 1 RandomChoice
    # transforms.RandomChoice([transforms.RandomVerticalFlip(p=1), transforms.RandomHorizontalFlip(p=1)]),

    # 2 RandomApply
    # transforms.RandomApply([transforms.RandomAffine(degrees=0, shear=45, fillcolor=(255, 0, 0)),
    #                         transforms.Grayscale(num_output_channels=3)], p=0.5),
    # 3 RandomOrder
    # transforms.RandomOrder([transforms.RandomRotation(15),
    #                         transforms.Pad(padding=32),
    #                         transforms.RandomAffine(degrees=0, translate=(0.01, 0.1), scale=(0.9, 1.1))]),

    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

# 构建MyDataset实例
train_data = RMBDataset(data_dir=train_dir, transform=train_transform)
valid_data = RMBDataset(data_dir=valid_dir, transform=valid_transform)

# 构建DataLoder
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=BATCH_SIZE)


# ============================ step 5/5 训练 ============================
for epoch in range(MAX_EPOCH):
    for i, data in enumerate(train_loader):

        inputs, labels = data   # B C H W

        img_tensor = inputs[0, ...]     # C H W
        img = transform_invert(img_tensor, train_transform)
        plt.imshow(img)
        plt.show()
        plt.pause(0.5)
        plt.close()

## 自定义transforms方法

**自定义transforms方法要素**

    transforms的内部实现：
    
        class Compose(object):
            
            def __call__(self,img):
                
                    for t in self.transforms:
                        img=t(img)
                    
                    return img
                    
- 仅接收一个参数，返回一个参数

- 注意上下游的输出与输入



**自定义transforms方法基本架构**


    class YourTransforms(object):
    
        def __init__(self,.....):  ##初始化，传入自定义参数
            pass
        
        def __call__(self,img):
            ....                    ##具体实现img功能
            
            return img
    

In [1]:
'''
添加椒盐噪声实例
'''

class AddPepperNoise(object):
    """增加椒盐噪声
    Args:
        snr （float）: Signal Noise Rate
        p (float): 概率值，依概率执行该操作
    """

    def __init__(self, snr, p=0.9):
        assert isinstance(snr, float) or (isinstance(p, float))
        self.snr = snr
        self.p = p

    def __call__(self, img):
        """
        Args:
            img (PIL Image): PIL Image
        Returns:
            PIL Image: PIL image.
        """
        if random.uniform(0, 1) < self.p:
            img_ = np.array(img).copy()
            h, w, c = img_.shape
            signal_pct = self.snr
            noise_pct = (1 - self.snr)
            mask = np.random.choice((0, 1, 2), size=(h, w, 1), p=[signal_pct, noise_pct/2., noise_pct/2.])
            mask = np.repeat(mask, c, axis=2)
            img_[mask == 1] = 255   # 盐噪声
            img_[mask == 2] = 0     # 椒噪声
            return Image.fromarray(img_.astype('uint8')).convert('RGB')
        else:
            return img

'''
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    AddPepperNoise(0.9, p=0.5),##自定义transforms方法的使用
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),])

'''

In [ ]:
'''
自定义transforms方法实例
'''

import os
import numpy as np
import torch
import random
import torchvision.transforms as transforms
from PIL import Image
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
from tools.my_dataset import RMBDataset
from tools.common_tools import transform_invert


def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)


set_seed(1)  # 设置随机种子

# 参数设置
MAX_EPOCH = 10
BATCH_SIZE = 1
LR = 0.01
log_interval = 10
val_interval = 1
rmb_label = {"1": 0, "100": 1}


class AddPepperNoise(object):
    """增加椒盐噪声
    Args:
        snr （float）: Signal Noise Rate
        p (float): 概率值，依概率执行该操作
    """

    def __init__(self, snr, p=0.9):
        assert isinstance(snr, float) or (isinstance(p, float))
        self.snr = snr
        self.p = p

    def __call__(self, img):
        """
        Args:
            img (PIL Image): PIL Image
        Returns:
            PIL Image: PIL image.
        """
        if random.uniform(0, 1) < self.p:
            img_ = np.array(img).copy()
            h, w, c = img_.shape
            signal_pct = self.snr
            noise_pct = (1 - self.snr)
            mask = np.random.choice((0, 1, 2), size=(h, w, 1), p=[signal_pct, noise_pct/2., noise_pct/2.])
            mask = np.repeat(mask, c, axis=2)
            img_[mask == 1] = 255   # 盐噪声
            img_[mask == 2] = 0     # 椒噪声
            return Image.fromarray(img_.astype('uint8')).convert('RGB')
        else:
            return img


# ============================ step 1/5 数据 ============================
split_dir = os.path.join("..", "..", "data", "rmb_split")
train_dir = os.path.join(split_dir, "train")
valid_dir = os.path.join(split_dir, "valid")

norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]


train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    AddPepperNoise(0.9, p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

# 构建MyDataset实例
train_data = RMBDataset(data_dir=train_dir, transform=train_transform)
valid_data = RMBDataset(data_dir=valid_dir, transform=valid_transform)

# 构建DataLoder
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=BATCH_SIZE)


# ============================ step 5/5 训练 ============================
for epoch in range(MAX_EPOCH):
    for i, data in enumerate(train_loader):

        inputs, labels = data   # B C H W

        img_tensor = inputs[0, ...]     # C H W
        img = transform_invert(img_tensor, train_transform)
        plt.imshow(img)
        plt.show()
        plt.pause(0.5)
        plt.close()


**总结：**

- 让训练集与测试集在经过变换后分布接近